<a href="https://colab.research.google.com/github/catafest/colab_google/blob/master/catafest_061.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


An open-source SDK for working with quantum computers at the level of extended quantum circuits, operators, and primitives.



See more on this webpage : https://pypi.org/project/qiskit/

In [12]:
!pip install qiskit
!pip install qiskit_ibm_runtime
!pip install qiskit[visualization]

In [14]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator


# first let's create a new circuit with two qubits
qc = QuantumCircuit(2)

# now add a Hadamard gate to qubit 0
qc.h(0)

# this perform a controlled-X gate on qubit 1, controlled by qubit 0
qc.cx(0, 1)
# draw the circuit
qc.draw()

┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘

In [18]:
# Set up six different observables.
observables_labels = ["ZZ", "ZI", "IZ", "XX", "XI"]
observables = [SparsePauliOp(label) for label in observables_labels]

In [22]:
# Save an IBM Cloud account on disk
# use your token from https://quantum.ibm.com/ here in token var
token = 'your_token_IBM'
QiskitRuntimeService.save_account(channel="ibm_quantum", token=token)# Save an IBM Cloud account on disk

service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.least_busy(operational=True, simulator=False)
backend.name

'ibm_sherbrooke'

In [25]:
# Run on the least-busy backend you have access to
backend = service.least_busy(simulator=False, operational=True)

# Convert to an ISA circuit and layout-mapped observables.
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(qc)

isa_circuit.draw('text', idle_wires=False)

global phase: 7π/4
          ┌────────┐ ┌────┐          ┌──────┐┌──────────┐┌────┐┌─────────┐
q_0 -> 0 ─┤ Rz(-π) ├─┤ √X ├──────────┤1     ├┤ Rz(-π/2) ├┤ √X ├┤ Rz(π/2) ├
         ┌┴────────┴┐├────┤┌────────┐│  Ecr │├─────────┬┘├────┤├─────────┤
q_1 -> 1 ┤ Rz(-π/2) ├┤ √X ├┤ Rz(-π) ├┤0     ├┤ Rz(π/2) ├─┤ √X ├┤ Rz(π/2) ├
         └──────────┘└────┘└────────┘└──────┘└─────────┘ └────┘└─────────┘

In [26]:
# Construct the Estimator instance we want to use.

estimator = Estimator(backend)
estimator.options.resilience_level = 1
estimator.options.default_shots = 5000

observables = [
    observable.apply_layout(isa_circuit.layout) for observable in observables
]

# One pub, with one circuit to run against five different observables.
job = estimator.run([(isa_circuit, observables)])

# This is the result of the entire submission.  We submitted one Pub,
# so this contains one inner result (and some metadata of its own).
job_result = job.result()

# This is the result from our single pub, which had five observables,
# so contains information on all five.
pub_result = job.result()[0]

In [32]:
print(job_result)


PrimitiveResult([PubResult(data=DataBin<5>(evs=[ 0.99093112  0.00326115 -0.00120486  0.99545592  0.00652229], stds=[0.00675069 0.01206515 0.01170717 0.00697005 0.01811151], ensemble_standard_error=[0.00703808 0.01444685 0.0142334  0.00693128 0.01444666]), metadata={'shots': 5024, 'target_precision': 0.01414213562373095, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32})], metadata={'dynamical_decoupling': {'enable': False, 'sequence_type': 'XX', 'extra_slack_distribution': 'middle', 'scheduling_method': 'alap'}, 'twirling': {'enable_gates': False, 'enable_measure': True, 'num_randomizations': 'auto', 'shots_per_randomization': 'auto', 'interleave_randomizations': True, 'strategy': 'active-accum'}, 'resilience': {'measure_mitigation': True, 'zne_mitigation': False, 'pec_mitigation': False}, 'version': 2})


In [33]:
print(pub_result)

PubResult(data=DataBin<5>(evs=[ 0.99093112  0.00326115 -0.00120486  0.99545592  0.00652229], stds=[0.00675069 0.01206515 0.01170717 0.00697005 0.01811151], ensemble_standard_error=[0.00703808 0.01444685 0.0142334  0.00693128 0.01444666]), metadata={'shots': 5024, 'target_precision': 0.01414213562373095, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32})


In [38]:
data = observables_labels
values = pub_result.data.evs


In [39]:
print("data = ", data)

data =  ['ZZ', 'ZI', 'IZ', 'XX', 'XI']


In [40]:
print("values = ", values)

values =  [ 0.99093112  0.00326115 -0.00120486  0.99545592  0.00652229]
